In [2]:
from util.data_loader import PhysioDataset
from util.transforms import filterBank
from torch.utils.data import DataLoader
from model.mixed_fbcnet import MIXED_FBCNet
from torch import nn
from torch.optim import lr_scheduler
from sklearn.metrics import det_curve

import os
import torch
import numpy as np
import yaml

We use the data loader and most code provided by the paper:

"Towards Enhanced EEG-based Authentication with Motor Imagery Brain-Computer Interface" by Bingkun Wu et al.

https://dl.acm.org/doi/abs/10.1145/3564625.3564656

# 1. Load the intra test data set

Here's the link for the dataset:

https://physionet.org/content/eegmmidb/1.0.0/

Make sure you download and install the dataset such that it can be accesible through a path that you define here:

In [5]:
path="./data/physionet/physionet.org/files/eegmmidb/1.0.0"

In [6]:
filterTransform = filterBank([[4,8],[8,12],[12,16],[16,20],[20,24],[24,28],[28,32],[32,36],[36,40]], 160) # frequency bands
    
batch_size = 4
        
channels =  [9, 14, 15, 16, 17, 18, 19, 21, 22, 62] # electrodes

subject = 3

intra_test_data = PhysioDataset(subject=subject, path=path, train="intra_test", transform=filterTransform, channels=channels, preprocess=False)

intra_test_dataloader = DataLoader(intra_test_data, batch_size=batch_size, shuffle=True, drop_last=True)

We wan to use the Intra test data set as it contains attackers that the system is already familiar with (also known as inside attackers).

By design, this dataset contains bona fide samples from the subject for which the model was trained (here set to subject 3), as well as sample from other subjects as attackers.

To see more details on this dataset, please check the paper.

In [7]:
type(intra_test_data)

util.data_loader.PhysioDataset

In [8]:
# Collect all samples and labels from the dataloader
all_samples = []
all_labels = []

for samples, labels in intra_test_data:
    all_samples.append(samples.unsqueeze(0))  # Add an extra batch dimension
    all_labels.append(labels)

# Convert lists to tensors
samples = torch.cat(all_samples, dim=0)
labels = torch.tensor(all_labels, dtype=torch.long)

# Printing the shapes to verify
print(samples.shape)
print(labels.shape)

torch.Size([125, 10, 640, 9])
torch.Size([125])


In [9]:
labels

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])

The label 1 means bona fide and 0 means attacker

In [10]:
samples.shape # each sample has 10 channels, 640 samples and 9 frequency bands, therefore this should be 125x10x640x9

torch.Size([125, 10, 640, 9])

# 2. Load the pretrained model

In [14]:
model_path= "./trained/3_99.2_98.4_20.pth"

In [15]:
model = MIXED_FBCNet(nChan=10, nBands=9) #class where the model is defined.
model.load_state_dict(torch.load(model_path)) # Load the weights
model.eval() # Set the model to evaluation mode

MIXED_FBCNet(
  (channelProj): Sequential(
    (0): Conv2dWithConstraint(10, 30, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (shapeTrans): Sequential(
    (0): Conv2dWithConstraint(30, 30, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (spatialConv): Sequential(
    (0): Conv2dWithConstraint(9, 288, kernel_size=(10, 1), stride=(1, 1), groups=9)
    (1): BatchNorm2d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (pointWise): Sequential(
    (0): Conv2dWithConstraint(288, 288, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(288, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): swish()
  )
  (varLayer): LogVarLayer()
  (standTemporalLayer): Sequential(
    (0): Dropout2d(p=0.5, inplace=Fals

# 3. Using the trained model to authenticate "Bona Fide" vs "Attacker" Scenarios (DEMO)

This is where we would use newly recorded data from an EEG device in order to test the strenght of our security system.

In [16]:
attacker = samples[50] # I jus took a sample for which the label was 0
bona_fide= samples[0] # same but here the label is 1

In [18]:
# Predict the class of the Attacker:
with torch.no_grad():
    output = model(attacker.unsqueeze(0))

print(f"Raw output: {output}")

# Convert raw output to predicted class index
predicted_class = output.argmax(dim=1)

print(f"Predicted class: {predicted_class.item()}")

print("--- Authentication Successful ---" if predicted_class.item() else "--- Authentication Failed ---")

Raw output: tensor([[-3.5089e-04, -7.9551e+00]])
Predicted class: 0
--- Authentication Failed ---
